In [3]:
from tkinter import *
from tkinter import ttk
import json
import requests
import csv
import sys
from tkinter import messagebox
from tkinter import PhotoImage
import tkinter as tk


class ScrollableFrame(ttk.Frame):
    def __init__(self, container, *args, **kwargs):
        super().__init__(container, *args, **kwargs)
        canvas = tk.Canvas(self)
        scrollbar = ttk.Scrollbar(self, orient="vertical", command=canvas.yview)
        self.scrollable_frame = ttk.Frame(canvas)

        self.scrollable_frame.bind(
            "<Configure>",
            lambda e: canvas.configure(
                scrollregion=canvas.bbox("all")
            )
        )

        canvas.create_window((0, 0), window=self.scrollable_frame, anchor="nw")

        canvas.configure(yscrollcommand=scrollbar.set)

        canvas.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")

class System:
    def __init__(self):
        print("Welcome to Group # EDFS! For more information or tutorial please use command \"gethelp\"")

    def mkdir(self, path):
        #TODO: Get user input command and create new Json Object at specified position
        iurl = "https://dsci551-f1a83-default-rtdb.firebaseio.com"
        paths = path[1:].split("/")
        path_string = ""
        for s in paths:
            path_string += "/" + s
            respond = requests.get(iurl + path_string + ".json")
            if respond.json() == None:
                data = {"Type":"Directory"}
                data = json.dumps(data)
                response = requests.put(iurl + path_string + ".json", data)
        messagebox.showinfo("Command Successful", "Directory Created")

    def ls(self, path):
        #TODO: Get user input command and show content at that path
        c_str = ""
        iurl = "https://dsci551-f1a83-default-rtdb.firebaseio.com"
        respond = requests.get(iurl + path + ".json")
        respond_json = respond.json()
        #print(respond_json)
        list_object = []
        for attribute, value in respond_json.items():
            if attribute != "Type":
                list_object.append(attribute)
        for i in list_object:
            c_str += i + " "
        messagebox.showinfo("Files in the path", c_str)
        #return (respond_json, list_object)
    
    def cat(self, path, k = 0):
        #TODO: Get user input command and show content of a file
        final_data = {}
        paths = path[1:].split("/")
        file_name = paths[-1]
        if path == "/":
            messagebox.showwarning("Warning", "This is a Directory")
            return
        iurl = "https://dsci551-f1a83-default-rtdb.firebaseio.com"
        dataurl = "https://dsci551-7dcf3-default-rtdb.firebaseio.com"
        respond = requests.get(iurl + path[:-4] + ".json")
        respond = respond.json()
        print(type(respond))
        if (respond == None):
            messagebox.showwarning("Warning", "The file can't be found")
            return
        else:
            if (respond["Type"] == "Directory"):
                messagebox.showwarning("Warning", "This is a Directory")
            # k = 0, display all
            # k = other, display Partition #k
            else:
                if (k == 0):
                    for res in respond:
                        if (res != "Type"):
                            respond_new = requests.get(respond[res])
                            final_data.update(respond_new.json())
                else:
                    if (k > len(respond) - 1):
                        messagebox.showwarning("Warning", "Partition # out of range. The maximum partition is: " + str(len(respond) - 1))
                        return
                    else:
                        respond_new = requests.get(respond["P" + str(k)])
                        final_data.update(respond_new.json())
                self.top = Toplevel() # new window
                list_attribute = []
                list_value = list(final_data.values())
                #print(list_value)
                temp = list_value[0]
                for key in temp.keys():
                    list_attribute.append(key)
                tuple_attribute = tuple(list_attribute)
                self.top.title(file_name)
                self.top.geometry("700x520")
                yscrlbar = ttk.Scrollbar(self.top, orient = "vertical")
                xscrlbar = ttk.Scrollbar(self.top, orient = "horizontal")
                self.trv = ttk.Treeview(self.top, columns = tuple_attribute, show="headings")
                self.trv.place(relx = 0.01, rely = 0.128, width = 646, height = 410)
                yscrlbar.place(relx = 0.934, rely = 0.128, width = 22, height = 432)
                xscrlbar.place(relx = 0.002, rely = 0.922, width = 651, height = 22)
                self.trv.configure(selectmode = "extended")
                self.trv.configure(xscrollcommand = xscrlbar.set, yscrollcommand = yscrlbar.set)
                yscrlbar.configure(command = self.trv.yview)
                xscrlbar.configure(command = self.trv.xview)
                for i in range(len(list_attribute)):
                    self.trv.heading(list_attribute[i], text = list_attribute[i], anchor = "center")
                    self.trv.column(list_attribute[i], width = 100, stretch= False)
                for item in list_value:
                    temp_list = []
                    for key in item.keys():
                        temp_list.append(item[key])
                    temp_list = tuple(temp_list)
                    #print(temp_list)
                    self.trv.insert("", 'end', text = "", values = temp_list)
                if k == 0:
                    if file_name == "insurance.csv":
                        Label(self.top, text = "Charge").place(relx = 0.01, rely = 0.05)
                        Label(self.top, text = "from").place(relx = 0.09, rely = 0.03)
                        Label(self.top, text = "to").place(relx = 0.09, rely = 0.07)
                        e1 = Entry(self.top, width = 50)
                        #e1.grid(row = 0, column = 0)
                        e1.place(relx = 0.14, rely = 0.03, width = 250, height = 20)
                        e2 = Entry(self.top, width = 50)
                        e2.place(relx = 0.14, rely = 0.07, width = 250, height = 20)
                        Button(self.top, text = "Search", command = lambda: self.pSearch(path, e1.get(), e2.get())).place(relx = 0.53, rely = 0.04)
                        Button(self.top, text = "Analytics", command = lambda: self.pAnalytics(path)).place(relx = 0.6, rely = 0.04)
                    elif file_name == "protein.csv":
                        Label(self.top, text = "Country").place(relx = 0.01, rely = 0.05)
                        e1 = Entry(self.top, width = 50)
                        e1.place(relx = 0.1, rely = 0.05, width = 250, height = 20)
                        Button(self.top, text = "Search", command = lambda: self.sSearch(path, e1.get())).place(relx = 0.5, rely = 0.04)
                        Button(self.top, text = "Analytics", command = lambda: self.sAnalytics(path)).place(relx = 0.6, rely = 0.04)
                        
                #trv.configure(xscrollcommand = verscrlbar.set)
                #verscrlbar.pack()

    def rm(self, path):
        #TODO: Get user input command and remove Json Object at specified position
        iurl = "https://dsci551-f1a83-default-rtdb.firebaseio.com"
        dataurl = "https://dsci551-7dcf3-default-rtdb.firebaseio.com"
        paths = path[1:].split("/")
        file_name = paths[-1][:-4]
        respond = requests.get(iurl + path[:-4] + ".json")
        if respond.json() == None:
            messagebox.showwarning("Warning", "Can't find the file")
            return
        else:
            if (respond.json()["Type"] == "File"):
                respond_data = requests.delete("https://dsci551-7dcf3-default-rtdb.firebaseio.com/" + file_name + ".json")
                respond_inode = requests.delete("https://dsci551-f1a83-default-rtdb.firebaseio.com" + path[:-4] + ".json")
            else:
                messagebox.showwarning("Warning", "It's not a file")
                return
        #respond = requests.delete("https://dsci551-7dcf3-default-rtdb.firebaseio.com/carsP2.json")
        messagebox.showinfo("Command Successful", "File Removed")
    
    def put(self, file, path, k):
        #TODO: Get user input command and upload file to the specified position  put(cars.csv, /user/john, k = # partitions)
        iurl = "https://dsci551-f1a83-default-rtdb.firebaseio.com"
        dataurl = "https://dsci551-7dcf3-default-rtdb.firebaseio.com/" + file[:-4]
        respond = requests.get(iurl + path + ".json")
        if respond.json() == None:
            #messagebox.showwarning("Warning","Can't find the path")
            return
        data = {}
        whole_data = {}
        positions = []
        length = 0
        count = 0
        index = 1
        try:
            with open(file, encoding='utf-8') as csvf:
                csvReader = csv.DictReader(csvf)
                for rows in csvReader:
                    length += 1
                p = length // k + 1 # avg length of each part
        except:
            return
        with open(file, encoding='utf-8') as csvf:
            csvReader = csv.DictReader(csvf)
            for rows in csvReader:

                if (index % p == 0):
                    count += 1
                    data = json.dumps(data)
                    position = dataurl + "/" + file[:-4] + "P" + str(count) + ".json"
                    response = requests.put(position, data)
                    positions.append(position)
                    data = {}
                if (index == length):
                    if file == 'insurance.csv':
                        key = rows['age'] + '_' + str(index)
                        rows['charges'] = float(rows['charges'])
                    elif file == 'protein.csv':
                        rows['Population'] = int(rows['Population'])
                        key = rows['Country']
                    else: 
                        rows['price'] = float(rows['price'])
                        key = "car" + rows['car_ID']
                    data[key] = rows
                    count += 1
                    data = json.dumps(data)
                    position = dataurl + "/" + file[:-4] + "P" + str(count) + ".json"
                    response = requests.put(position, data)
                    positions.append(position)
                    data = {}
                if file == 'insurance.csv':
                    key = rows['age'] + '_' + str(index)
                    rows['charges'] = float(rows['charges'])
                elif file == 'protein.csv':
                    rows['Population'] = float(rows['Population'])
                    key = rows['Country']
                else: 
                    rows['price'] = float(rows['price'])
                    key = "car" + rows['car_ID']
                data[key] = rows
                index += 1       
        data = {}
        data["Type"] = "File"
        for i in range(len(positions)):
            data["P"+str(i+1)] = positions[i]
        data = json.dumps(data)
        response = requests.put(iurl + path + "/" + file[:-4] + ".json", data)
        #messagebox.showinfor("put completed")

    

    def getPartitionLocations(self, path):
        iurl = "https://dsci551-f1a83-default-rtdb.firebaseio.com"
        respond = requests.get(iurl + path[:-4] + ".json")
        respond = respond.json()
        location_string = ""
        if (respond == None):
            print("The file can't be found")
        else:
            if (respond["Type"] == "Directory"):
                messagebox.showwarning("Warning", "This is not a file")
            else:
                for res in respond:
                    if (res != "Type"):
                        location_string += respond[res] + "\n"
                messagebox.showinfo("Partition Locations", location_string)
                
                        
    
    def readPartition(self, path, partition):
        iurl = "https://dsci551-f1a83-default-rtdb.firebaseio.com"
        dataurl = "https://dsci551-7dcf3-default-rtdb.firebaseio.com"
        respond = requests.get(iurl + path[:-4] + ".json")
        respond = respond.json()
        if (respond == None):
            print("The file can't be found")
        else:
            if (respond["Type"] == "Directory"):
                print("This is a Directory")
            else:
                respond_new = requests.get(respond["P" + str(partition)])
                return(respond_new.json())
        print("readPartition")
        
    def sSearch(self, path, f1):
        total = 0
        num_list = []
        data = parallelSearch(path, f1)
        for item in self.trv.get_children():
            self.trv.delete(item)
        for item in list(data.values()):
            num = 0
            for i in list(item.values()):
                temp_list = []
                for key in i.keys():
                    temp_list.append(i[key])
                temp_list = tuple(temp_list)
                #print(temp_list)
                self.trv.insert("", 'end', text = "", values = temp_list)
                num += 1
                total += 1
            num_list.append(num)
        top1 = Toplevel()
        row_c = 2
        Label(top1, text = "The Search function find the country with name containing the string input").grid(row = 0, column = 0)
        Label(top1, text = "Mapping:").grid(row = 1, column = 0, sticky ="w")
        for i in list(data.keys()):
            Label(top1, text = "Partition " + i + ": " + str(num_list[int(i)-1]) + "results found").grid(row = row_c, column = 0, sticky = "w")
            row_c += 1
        Label(top1, text = "Reducing:").grid(row = row_c, column = 0, sticky ="w")
        Label(top1, text = "The total number of research found is: " + str(total)).grid(row = row_c + 1, column = 0, sticky = "w")

    def pSearch(self, path, f1, f2):
        if f1 == "":
            f1 = 0
        if f2 == "":
            f2 = 100000000000000
        try:
            float(f1)
        except:
            messagebox.showwarning("value error", "range is not a numeric")
            return
        try:
            float(f2)
        except:
            messagebox.showwarning("value error", "range is not a numeric")
            return
        fr = float(f1)
        to = float(f2)
        total = 0
        num_list = []
        data = parallelSearch(path, fr, to)
        for item in self.trv.get_children():
            self.trv.delete(item)
        for item in list(data.values()):
            num = 0
            for i in list(item.values()):
                temp_list = []
                for key in i.keys():
                    temp_list.append(i[key])
                temp_list = tuple(temp_list)
                #print(temp_list)
                self.trv.insert("", 'end', text = "", values = temp_list)
                num += 1
                total += 1
            num_list.append(num)
        top1 = Toplevel()
        row_c = 2
        Label(top1, text = "The Search function find the insurance charge within a range").grid(row = 0, column = 0)
        Label(top1, text = "Mapping:").grid(row = 1, column = 0, sticky ="w")
        for i in list(data.keys()):
            Label(top1, text = "Partition " + i + ": " + str(num_list[int(i) - 1]) + "results found").grid(row = row_c, column = 0, sticky = "w")
            row_c += 1
        Label(top1, text = "Reducing:").grid(row = row_c, column = 0, sticky ="w")
        Label(top1, text = "The total number of research found is: " + str(total)).grid(row = row_c + 1, column = 0, sticky = "w")
        
            
    def pAnalytics(self, path):
        data = parallelAnalytics(path)
        top1 = Toplevel()
        row_c = 2
        smoker = 0
        nonsmoker = 0
        Label(top1, text = "The analytics function find the sum of insurance charge for male smoker and male nonsmoker").grid(row = 0, column = 0)
        Label(top1, text = "Mapping:").grid(row = 1, column = 0, sticky ="w")
        for i in list(data.keys()):
            Label(top1, text = "Partition " + i + ": The Total charges of male smoker is " + str(data[i]["smoke"]) + ": The Total charges of male nonsmoker is " + str(data[i]["nosmoke"])).grid(row = row_c, column = 0, sticky = "w")
            row_c += 1
            smoker += data[i]["smoke"]
            nonsmoker += data[i]["nosmoke"]
        Label(top1, text = "Reducing:").grid(row = row_c, column = 0, sticky ="w")
        Label(top1, text = "The total sum of insurance charge for male smoker is: " + str(smoker) + " for male nonsmoker is: " + str(nonsmoker)).grid(row = row_c + 1, column = 0, sticky = "w")
        
        
        
    def sAnalytics(self, path):
        data = parallelAnalytics(path)
        top1 = Toplevel()
        row_c = 2
        max_obesity = 0
        max_country = ""
        Label(top1, text = "The analytics function find the country which has a maximum Covid confirmed rate").grid(row = 0, column = 0)
        Label(top1, text = "Mapping:").grid(row = 1, column = 0, sticky ="w")
        for i in list(data.keys()):
            Label(top1, text = "Partition " + i + ": The country which has a maximum confirmed rate is " +  str(data[i]["country"]) + " (" + str(data[i]["obesity"]) + ")").grid(row = row_c, column = 0, sticky = "w")
            row_c += 1
            max_obesity = max(max_obesity, data[i]["obesity"])
            if (data[i]["obesity"] >= max_obesity):
                max_country = data[i]["country"]
        Label(top1, text = "Reducing:").grid(row = row_c, column = 0, sticky ="w")
        Label(top1, text = "The country with maximum confirmed rate among partitions is: " + str(max_country)+ " (" + str(max_obesity) + ")").grid(row = row_c + 1, column = 0, sticky = "w")
        

    # # a callback function to merge data
    # def collect_result(result):
    #     global results
    #     results.update(result)

if __name__ == "__main__":
    s = System()
    last_location = []
    def analyticsPartition(file, part, i):
        ret = {}
        ret[str(i)] = {}
        max_obesity = 0
        max_country = ""
        insurance_smoke = 0
        insurance_nosmoke = 0
        # find the sum of insurance charges for male smokers and male nonsmokers
        if file == 'insurance.csv':
            # find the searched range
            for key, value in part.items():
                charge = value["charges"]
                sex = value["sex"]
                smoke = value["smoker"]
                if smoke == "yes":
                    if sex == "male":
                        insurance_smoke += charge
                else:
                    if sex == "male":
                        insurance_nosmoke += charge
            ret[str(i)]["smoke"] = insurance_smoke
            ret[str(i)]["nosmoke"] = insurance_nosmoke
        elif file == 'protein.csv':
            for key, value in part.items():
                country = value["Country"]
                if value['Confirmed'] == "NA":
                    continue
                obesity = float(value['Confirmed'])
                if obesity > max_obesity:
                    max_country = country
                    max_obesity = obesity
            ret[str(i)]["country"] = max_country
            ret[str(i)]["obesity"] = max_obesity
        else:
            print('Error')
        return ret
    
    def parallelAnalytics(path):
        iurl = "https://dsci551-f1a83-default-rtdb.firebaseio.com"
        paths = path[1:].split("/")
        file_name = paths[-1]
        respond = requests.get(iurl + path[:-4] + ".json")
        respond = respond.json()
        ret = {}
        if (respond == None):
            messagebox.showwarning("Warning", "The file can't be found")
            return
        else:
            if (respond["Type"] == "Directory"):
                messagebox.showwarning("Warning", "This is a Directory")
            else:
                k = len(respond) - 1
                for i in range(1, k+1):
                    new = analyticsPartition(file_name, s.readPartition(path, i), i)
                    ret.update(new)
        return ret
    
    def searchPartition(file, part, i, *args):
        ret = {}
        ret[str(i)] = {}
        if file == 'insurance.csv':
            minimum, maximum = args
            # find the searched range
            for key, value in part.items():
                charge = value['charges'] 
                if charge >= minimum and charge <= maximum:
                    ret[str(i)][key] = value
                    # print(key, value)
        elif file == 'protein.csv':
            name = args[0]
            for key, value in part.items():
                country = value['Country']
                if name in country: # contains string
                    ret[str(i)][key] = value
        else:
            print('Error')
        return ret
    
    
    def parallelSearch(path, *args):
        iurl = "https://dsci551-f1a83-default-rtdb.firebaseio.com"
        paths = path[1:].split("/")
        file_name = paths[-1]
        respond = requests.get(iurl + path[:-4] + ".json")
        respond = respond.json()
        ret = {}
        if (respond == None):
            messagebox.showwarning("Warning", "The file can't be found")
            return
        else:
            if (respond["Type"] == "Directory"):
                messagebox.showwarning("Warning", "This is a Directory")
            else:
                k = len(respond) - 1
                for i in range(1, k+1):
                    new = searchPartition(file_name, s.readPartition(path, i), i, *args)
                    ret.update(new)
        return ret
    
    def get_info(path):
        iurl = "https://dsci551-f1a83-default-rtdb.firebaseio.com"
        respond = requests.get(iurl + path + ".json")
        respond_json = respond.json()
        list_object = []
        for attribute, value in respond_json.items():
            if attribute != "Type":
                list_object.append(attribute)
        return (respond_json, list_object)
    
    def back():
        global file_btimg
        global current_location
        global last_location
        if len(last_location) == 0:
            return
        current_location = last_location[-1]
        try:
            if (current_location == "/"):
                current_json, current_directories = get_info(current_location)
            else:    
                current_json, current_directories = get_info(current_location[:-1])
        except:
            return
        last_location.pop(-1)
        location_Label.config(text = location_string + current_location)
        for widget in frame.scrollable_frame.winfo_children():
            widget.destroy()
        for i in current_directories:
            if "Type" not in current_json[i]:
                return
            if current_json[i]["Type"] == "Directory":
                Button(frame.scrollable_frame, image = file_btimg, borderwidth = 0, command = lambda i = i: enter_directory(i)).pack()
            else:
                Button(frame.scrollable_frame, image = csv_btimg, borderwidth = 0, command = lambda i = i: enter_directory(i)).pack()
            Label(frame.scrollable_frame, text = i, font='Helvetica 8 bold').pack()

    def enter_directory(path):
        global csv_btimg
        global file_btimg
        global current_location
        global last_location
        temp = current_location
        new_location = temp + path + "/"
        try:
            if (current_location == "/"):
                current_json, current_directories = get_info(new_location)
            else:
                current_json, current_directories = get_info(new_location[:-1])
        except:
            return
        if current_json["Type"] == "File":
            s.cat(new_location[:-1] + ".csv")
            print(current_location)
            return
        current_location = current_location + path + "/"
        last_location.append(temp)
        location_Label.config(text = location_string + current_location)
        for widget in frame.scrollable_frame.winfo_children():
            widget.destroy()
        for i in current_directories:
            if "Type" not in current_json[i]:
                return
            if current_json[i]["Type"] == "Directory":
                Button(frame.scrollable_frame, image = file_btimg, borderwidth = 0, command = lambda i = i: enter_directory(i)).pack()
            else:
                Button(frame.scrollable_frame, image = csv_btimg, borderwidth = 0, command = lambda i = i: enter_directory(i)).pack()
            Label(frame.scrollable_frame, text = i, font='Helvetica 8 bold').pack()
            
    
        
        
    def enter_command():
        command = e.get()
        if command == "":
            e.delete(0, END)
            return
        command = command.split()
        c = command[0]
        if c == "quit":
            root.destroy()
        elif c == "mkdir":
            s.mkdir(command[1])
            e.delete(0, END)
        elif c == "ls":
            s.ls(command[1])
            e.delete(0, END)
        elif c == "cat":
            s.cat(command[1])
            e.delete(0, END)
        elif c == "rm":
            s.rm(command[1])
            e.delete(0, END)
        elif c == "put":
            s.put(command[1], command[2], int(command[3]))
            e.delete(0, END)
        elif c == "gethelp":
            messagebox.showinfo("help", "The system supports the following commands:\no mkdir: create a directory in file system, e.g., mkdir /user/john\no ls: listing content of a given directory, e.g., ls /user\no cat: display content of a file, e.g., cat /user/john/hello.csv\no rm: remove a file from the file system, e.g., rm /user/john/cars.csv\no put: uploading a file to file system with k partitions, e.g., put hello.csv /user/john 3\no getPartitionLocations return the locations of partitions of the file, e.g., getPartitionLocations /user/john/cars.csv\no readPartition return the content of partition # of the specified file, e.g., readPartition /user/john/cars.scv 2")
            e.delete(0, END)
        elif c == "getPartitionLocations":
            s.getPartitionLocations(command[1])
            e.delete(0, END)
        elif c == "readPartition":
            s.cat(command[1], int(command[2]))
            e.delete(0, END)
        else:
            messagebox.showinfo("Attention", "No such command")
            e.delete(0, END)
    root = Tk()
    root.title("Mileo Sytem")
    root.config(bg = '#D3D3D3')
    inforLabel = Label(root, text = "Welcome to use Mileo Sytem! For more information or tutorial please use command \"gethelp\"", bg = "#D3D3D3")
    current_location = "/"
    inforLabel.grid(row = 0, column = 0)
    command_Label = Label(root, text = "Please put your command here: ", bg = "#D3D3D3")
    command_Label.grid(row = 1, column = 0, pady = 2)
    e = Entry(root, width = 50)
    e.grid(row = 2, column = 0, pady = 2)
    enter_button = Button(root, text = "Enter Command", command = enter_command)
    enter_button.grid(row = 3, column = 0, pady = 10)
    location_Label = Label(root, text = "Current location: " + current_location, font='Helvetica 13 bold', bg = "#D3D3D3")
    location_Label.grid(row = 5, column = 0)
    back_button = Button(root, text = "<", command = back)
    back_button.grid(row = 5, column = 0, sticky = W, padx = 50)
    frame = ScrollableFrame(root)
    location_string = "Current location: "
    #for i in range(50):
    #    ttk.Label(frame.scrollable_frame, text="Sample scrolling label").pack()
    #frame = Frame(root, bg = "#D3D3D3", bd = 10, height = 400, width = 400)
    frame.grid(row = 6, column = 0)
    initial_path = "https://dsci551-f1a83-default-rtdb.firebaseio.com/"
    init_json, init_directories = get_info("/")
    #file_btimg = PhotoImage(file = 'Folder Attempt.png')
    #button = Button(root, image = file_btimg, text = i, borderwidth = 1)
    #button.grid(row = 7, column = 0)
    x = 0
    y = 0
    file_btimg = PhotoImage(file = 'Assets/Folder_Attempt.png')
    csv_btimg = PhotoImage(file = 'Assets/128x128.png')
    for i in init_directories:
        if init_json[i]["Type"] == "Directory":
            Button(frame.scrollable_frame, image = file_btimg, borderwidth = 0, command = lambda i = i: enter_directory(i)).pack()
        else:
            Button(frame.scrollable_frame, image = csv_btimg, borderwidth = 0, command = lambda i = i: enter_directory(i)).pack()
        Label(frame.scrollable_frame, text = i, font='Helvetica 8 bold').pack()
    #location_Label = Label(frame, text = "Current location: " + current_location, font='Helvetica 6 bold')
    #location_Label.grid(row = 0, column = 0)
    root.mainloop()

Welcome to Group # EDFS! For more information or tutorial please use command "gethelp"


TypeError: string indices must be integers